
# *Handling Data*


Once you've read your data into R, you want to look at it in different ways to understand what you have. In this example, we will read in patient data and perform some basic data quality tasks and go through ways we can format the data. Although most of the examples that follow use small datasets, we will mainly use the syntax from the "data.table" package because it is very fast for large datasets.

In [8]:
library(data.table)
patients <- fread("test_big.csv")
head(patients)
# nb. data.table by default will show the first and last 5 rows of the dataset 

V1,patid,startdate,enddate,ht
1,1000,2016-11-30,2017-08-31,0
2,1001,2016-11-30,2017-03-04,0
3,1002,2016-11-30,2017-05-16,1
4,1003,2016-11-30,2017-10-11,1
5,1004,2016-11-30,2017-09-03,0
6,1005,2016-11-30,2017-01-22,1


There are two obvious issues here, the first is missing  values and non-existent dates ("0000-00-00"). We can fix these by extending what R considers a missing value when it reads the file in. To do this we supply additional values in the "na.strings" component of the command to read in the data:

In [3]:
patients <- fread("test_big.csv",
                na.strings = c("", " ", "0000-00-00"))

Here we are telling R to assign an NA (missing) value whereever it finds no value (""), a single space (" "), or impossible dates ("0000-00-00")

Normally the first question you want to ask is how is my data interpretted by R, to do this we can look at the class of each column:

In [9]:
sapply(patients, class)

V1       patid   startdate     enddate          ht 
"character" "character" "character" "character" "character"

Three columns ("rownumber", "patid", "ht") are all intergers as we might have expected from the column names. But we have a problem with the date columns because R they are of little use to use in a character string format. The fix is to convert them from character into the date storage type, fortunately this is very easy to do but first we will cover the basics of how R handles dates and times


## *Dates and Times*


Dates and times are complicated by months of different lengths, daylight saving, timezones, and leap years. An important part of analysing health records is good command of dates and times, we start from the absolute basics by asking R for the time like so:

In [10]:
Sys.time()

[1] "2017-11-30 11:54:05 GMT"

From left to right we are given the date as year, month, day; the time (24-hour clock) as hour, minute, second; and finally the timezone. Both the user and R understand this representation but in order to use it in calculations we need a numeric form. The industry standard used by nearly all systems is the POSIX  (Portable Operating System Interface) system, it stores dates and times as the number of seconds ellapsed from midnight on 1st January 1970.

To show the number of seconds between 1st January 1970 and today, just use the as.numeric function:

In [11]:
as.numeric(Sys.time())

[1] 1512042847

It should now be clear the first thing we need to do with the date columns is to convert them into the form R understands. We do this by using the as.Date function, the function will accept a character string representation of the date and requires us to tell it how the date is formatted, for example:

In [12]:
as.Date("14/4/2016", "%d/%m/%Y")

[1] "2016-04-14"

We use conversion specifications ('%') to seperate each component of the date, here into day ('d'), month ('m'), and year ('Y). Left blank the function will use the default of '%Y-%m-%d %H:%M:%S', which in our case, would not be correct. We can accept dates in any format thanks to this flexibility and there is an extensive catalogue of representations to please any taste, some examples are given below:


Conversion | Description
------------- | -------------
%A	|	Full weekday name
%B	|	Full month name
%d	|	Day of the month as decimal number (01–31)
%m	|	Month as decimal number (0–11)
%U	|	Week of the year (00–53) using the first Sunday as day 1 of week 1
%H	|	Hours as decimal number (00–23) on the 24-hour clock
%I	|	Hours as decimal number (01–12) on the 12-hour clock
%M	|	Minute as decimal number (00–59)
%S	|	Second as decimal number (00–61, allowing for two ‘leap seconds’)
%T	|	Equivalent to %H:%M:%S.

Going back to our example dataset, we convert date column from character into a date storage type.

In [14]:
patients[,startdate := as.Date(startdate, '%Y-%m-%d')]

Here we overwrite the "first_reg date", this is ok if you are sure of the conversion but when you first start out it is better to create a new column.

In [15]:
patients[,startdate_2 := as.Date(startdate, '%Y-%m-%d')]

Once you are statisified with the conversion, you can delete the original column and rename the new column

In [16]:
patients[,startdate := NULL] # assigning "NULL" to the column deletes the column
setnames(patients, old = "startdate_2", new = "startdate") 
# we can omit the "old/new = " part but included for clarity

To do this for several columns will become laborious. What we want to be able to do is tell R to convert all the columns that store dates as character strings into a date class. We will do this by using two new concepts - regular expressions and functions:

In [17]:
patients[,(grep("date", colnames(patients))) := 
           lapply(.SD, function(x) as.Date(x, '%Y-%m-%d')),.SDcols=grep("date", colnames(patients))]

Why do we invest so much time in getting dates into the correct storage type. The simple answer is to use them in calculations. Take the following example:

In [20]:
patients[, LoS := enddate - startdate]

In [22]:
head(patients)

V1,patid,enddate,ht,startdate,LoS
1,1000,2017-08-31,0,2016-11-30,274 days
2,1001,2017-03-04,0,2016-11-30,94 days
3,1002,2017-05-16,1,2016-11-30,167 days
4,1003,2017-10-11,1,2016-11-30,315 days
5,1004,2017-09-03,0,2016-11-30,277 days
6,1005,2017-01-22,1,2016-11-30,53 days



## *Text manipulation*


Medical diagnasoses and procedures provide an excellent example of handling non-numeric values in analysing
electronic healthcare records. Most if not all diagnoses and procedures are represented by a specific ICD-10 
code, these can be between three and seven characters long in a combination of numbers and characters.

All ICD-10 start with a letter followed by two numbers, this identifies the category of the code, for example
there are over 1,200 codes that relate to diseases of the circulatory system. The next three positions
tell us about the cause, site, and severity. For some codes there is a seventh positions that provides 
information of the type of encounter, i.e. first or subsequent for a particular issue.

Because of this combination of text and numbers (alphanumeric) it will be treated as a factor or character depending on the package you use. We have seen
how factors can be converted into character strings earlier so will start this section by exploring a simple
character string.

In [ ]:
icd10 <- fread("icd10cm_order_2016.txt")
cholera <- icd10$Description[1:4]
cholera

Here we have created a character vector by selecting the first four values of the description column 
but if we didn't know how many values are in a character vector, we can use the length function to find 
out; one example where you would use this is to count the number of GPs by CCG.

In [ ]:
length(cholera)

To count the number of characters in each element of the vector we use the nchar function, returning
a value for each element of the vector:

In [ ]:
nchar(cholera)

This is handy when what you want to do depends on the length of the output. Say we wanted to know how
many ICD10 codes have only three characters, i.e. codes that are not further specified beyond category
of the condition, we can count the number of such conditions contained within our dataset.

In [ ]:
length(nchar(icd10$Code) == 3)

What if we wanted to know how many subdivision of a particular category exist? We know the first three
character identify the category so we could use this to extract the information. If we take a single
character string as an example, we can use the substring function (substr):

In [4]:
substr("H4010X0", 1, 3)

[1] "H40"

The function takes your character string or vector as the first arguement and and selects all 
characters between the second (start) and third (end) arguements. In this case it returns a string 
containing the characters betweens 1st and 3rd positions of the string. We can use this to create a 
new variable and count the number of ICD-10 categories:

In [ ]:
icd10 <- cbind(icd10, Category = substr(icd10$Code, 1,3))
length(unique(icd10$Category))

A closely related function is 'substring', whereby you specify just the start position and it returns a string
from that position to the end of string by default. But of course you can use either function to get
the same result:

In [5]:
substring("H4010X0", 2)

[1] "4010X0"

In [6]:
substring("H4010X0", 1,3)

[1] "H40"

In [7]:
substr("H4010X0", 1,3)

[1] "H40"